In [1]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#train_input_dir = 'F://Taiba//FoR_limited sample//Combined//training//'
train_output_dir = 'F://Taiba//FoR_limited sample//Combined//training_spectrograms//'
#val_input_dir = 'F://Taiba//FoR_limited sample//2seconds//validation//'
#val_output_dir = 'F://Taiba//FoR_limited sample//2seconds//validation_spectrograms//'
#test_input_dir = 'F://Taiba//FoR_limited sample//Combined//testing//'
test_output_dir = 'F://Taiba//FoR_limited sample//Combined//testing_spectrograms//'


In [3]:
def generate_spectrogram(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.wav'):
                audio_path = os.path.join(root, file)
                y, sr = librosa.load(audio_path, sr=None)
                S = librosa.feature.melspectrogram(y=y, sr=sr)
                S_dB = librosa.power_to_db(S, ref=np.max)

                plt.figure(figsize=(10, 4))
                librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
                plt.colorbar(format='%+2.0f dB')
                plt.tight_layout()

                class_label = os.path.basename(root)
                save_dir = os.path.join(output_dir, class_label)
                os.makedirs(save_dir, exist_ok=True)
                save_file = os.path.splitext(file)[0] + '.png'
                save_file_path = os.path.join(save_dir, save_file)
                plt.savefig(save_file_path)
                plt.close()


In [4]:
#generate_spectrogram(train_input_dir, train_output_dir)
generate_spectrogram(val_input_dir, val_output_dir)
#generate_spectrogram(test_input_dir, test_output_dir)


NameError: name 'val_input_dir' is not defined

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_output_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

#val_generator = val_datagen.flow_from_directory(
    #val_output_dir,
    #target_size=(224, 224),
    #batch_size=32,
    #class_mode='binary'
#)

test_generator = test_datagen.flow_from_directory(
    test_output_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 23238 images belonging to 2 classes.
Found 11406 images belonging to 2 classes.


In [5]:
input_shape = (224, 224, 3)

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, epochs=20)

test_loss, test_accuracy = model.evaluate(test_generator)
print('Test accuracy:', test_accuracy)

train_loss, train_accuracy = model.evaluate(train_generator)
print('Train accuracy:', train_accuracy)


Epoch 1/20
727/727 [==============================] - 338s 460ms/step - loss: 0.3465 - accuracy: 0.8276
Epoch 2/20
727/727 [==============================] - 187s 257ms/step - loss: 0.1209 - accuracy: 0.9541
Epoch 3/20
727/727 [==============================] - 126s 173ms/step - loss: 0.0785 - accuracy: 0.9715
Epoch 4/20
727/727 [==============================] - 126s 173ms/step - loss: 0.0609 - accuracy: 0.9777
Epoch 5/20
727/727 [==============================] - 127s 174ms/step - loss: 0.0455 - accuracy: 0.9842
Epoch 6/20
727/727 [==============================] - 129s 177ms/step - loss: 0.0357 - accuracy: 0.9876
Epoch 7/20
727/727 [==============================] - 125s 171ms/step - loss: 0.0304 - accuracy: 0.9897
Epoch 8/20
727/727 [==============================] - 124s 171ms/step - loss: 0.0215 - accuracy: 0.9920
Epoch 9/20
727/727 [==============================] - 124s 170ms/step - loss: 0.0188 - accuracy: 0.9930
Epoch 10/20
727/727 [==============================] - 124s 170m